**Hyperparameter tuning**:

- It is the process of finding the optimal set of hyperparameters for a machine learning model.


**Key Concepts:**

*Hyperparameters vs. Parameters:*

- Parameters are learned during model training (e.g., weights in a neural network, coefficients in linear regression).

- Hyperparameters are set before the learning process and control how the model learns (e.g., learning rate, number of trees in a random forest, regularization strength).

In [30]:
from sklearn import svm, datasets

In [31]:
iris = datasets.load_iris()

In [32]:
import pandas as pd
df = pd.DataFrame(iris.data, columns=iris.feature_names)

df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


Approach 1: Use train_test_split and manually tune parameters by trial and error

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

In [34]:
model = svm.SVC(kernel='rbf', C = 30, gamma = 'scale')
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9333333333333333

Approach 2: Use K Fold Cross validation

In [35]:
from sklearn.model_selection import cross_val_score

In [36]:
cross_val_score(svm.SVC(kernel='linear', C = 10, gamma = 'auto'), iris.data, iris.target, cv = 5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [37]:
cross_val_score(svm.SVC(kernel='rbf', C = 10, gamma = 'auto'), iris.data, iris.target, cv = 5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [38]:
cross_val_score(svm.SVC(kernel='rbf', C = 20, gamma = 'auto'), iris.data, iris.target, cv = 5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [39]:
#Above approach is tiresome and very manual. We can use for loop as an alternative

import numpy as np
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': np.float64(0.9800000000000001),
 'rbf_10': np.float64(0.9800000000000001),
 'rbf_20': np.float64(0.9666666666666668),
 'linear_1': np.float64(0.9800000000000001),
 'linear_10': np.float64(0.9733333333333334),
 'linear_20': np.float64(0.9666666666666666)}

Approach 3: Use **GridSearchCV**

In [40]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(svm.SVC(gamma = 'auto'), {
    'C': [1,10,20],
    'kernel': ['rbf', 'linear']
}, cv=5, return_train_score=False) 


clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00162535, 0.00228434, 0.00102773, 0.00146279, 0.00215073,
        0.00153594]),
 'std_fit_time': array([6.50514935e-04, 5.46039186e-04, 1.53305927e-05, 6.21238203e-04,
        4.88161220e-04, 5.36370154e-04]),
 'mean_score_time': array([0.00142975, 0.00123158, 0.00166311, 0.00099277, 0.00126014,
        0.0014668 ]),
 'std_score_time': array([0.00039227, 0.00062167, 0.00023865, 0.00051996, 0.00049303,
        0.00056476]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value=np.str_('?'),
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 

exporting results  --->  Pandas Dataframe

In [41]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001625,0.000651,0.001430,0.000392,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.002284,0.000546,0.001232,0.000622,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001028,0.000015,0.001663,0.000239,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001463,0.000621,0.000993,0.000520,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.002151,0.000488,0.001260,0.000493,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.001536,0.000536,0.001467,0.000565,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [42]:
df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [43]:
clf.best_score_

np.float64(0.9800000000000001)

In [44]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

**RandomizedSearch CV**

- Use RandomizedSearchCV to reduce number of iterations and with random combination of parameters. This is useful when you have too many parameters to try and your training time is longer. It helps reduce the cost of computation

In [45]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2
)
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,20,rbf,0.966667
1,10,rbf,0.980000
